In [1]:
import ast
import json
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import StrMethodFormatter
from matplotlib.dates import DateFormatter
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
from sklearn import metrics 
import defi.defi_tools as dft


pools_daily = pd.read_csv('Uniswap Extractooor (1).csv', delimiter=';', encoding='utf-8')


# Sort by date
pools_daily['Date'] = pd.to_datetime(pools_daily['Date']).dt.strftime('%Y-%m-%d')
pools_daily.sort_values(by='Date (Unix)', inplace=True, ascending=True)
pools_daily.index = pools_daily['Date']


# Preprocessing, labelling units
pools_daily['Liquidity'] = pools_daily['Liquidity'].str.replace(',', '').astype(float)
pools_daily['Sqrt Price'] = pools_daily['Sqrt Price'].str.replace(',', '').astype(float)

pools_daily.rename(columns={'Token 0 Price': 'Token0 Price (WBTC)', 
                            'Token 1 Price': 'Token1 Price (WETH)',
                            'Volume Token 0': 'Volume Token 0 (WBTC)',
                            'Volume Token 1': 'Volume Token 1 (WETH)',
                            'Open': 'Open (WBTC)', 
                            'High': 'High (WBTC)', 
                            'Low': 'Low (WBTC)', 
                            'Close': 'Close (WBTC)'}, inplace=True)

pools_daily.dropna(inplace=True)
pools_daily


FileNotFoundError: [Errno 2] No such file or directory: 'Uniswap Extractooor (1).csv'

In [ ]:
# Construct dataframe with Impermanent Loss
pools_daily['Impermanent Loss (Stable Coin)'] = 0
pools_daily['Impermanent Loss (Token)'] = 0

for i in range(0, len(pools_daily)):
    ratio = pools_daily['Token1 Price Change'].iloc[i]
    price_ratio = pools_daily['Price Ratio'].iloc[i]
    
    if ratio > 1:
        pools_daily['Impermanent Loss (Stable Coin)'].iloc[i] = -1 * dft.iloss(ratio, numerical=True)
    else:
        pools_daily['Impermanent Loss (Stable Coin)'].iloc[i] = dft.iloss(ratio, numerical=True)
        
    if price_ratio > 1:
        pools_daily['Impermanent Loss (Token)'].iloc[i] = -1 * dft.iloss(price_ratio, numerical=True)
    else:
        pools_daily['Impermanent Loss (Token)'].iloc[i] = dft.iloss(price_ratio, numerical=True)
    


C:\Users\yinki\AppData\Local\Temp\ipykernel_27388\2959182047.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pools_daily['Impermanent Loss (Stable Coin)'].iloc[i] = -1 * dft.iloss(ratio, numerical=True)
C:\Users\yinki\AppData\Local\Temp\ipykernel_27388\2959182047.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pools_daily['Impermanent Loss (Token)'].iloc[i] = dft.iloss(price_ratio, numerical=True)


In [ ]:
# Adding Virtual Price = 2 x Liquidity / Sqrt Price and Virtual Price Change
pools_daily['Virtual Price'] = 2 * pools_daily['Liquidity'] / np.sqrt(pools_daily['Token0 Price (USDC)'] * pools_daily['Token1 Price (WETH)'])
pools_daily['Virtual Price Change'] = pools_daily['Virtual Price'].pct_change() + 1


# Adding Effective Fee Token1 Price - Token0 Price / Token0 Price
pools_daily['Effective Fee'] = (pools_daily['Token1 Price (WETH)'] - pools_daily['Token0 Price (USDC)']) / pools_daily['Token0 Price (USDC)']



pools_daily.reset_index(inplace=True, drop=True)
pools_daily.to_csv('pools_daily_ILoss.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
pools_daily

,Date,ID,Date (Unix),Pool,Pool Tokens,Liquidity,Sqrt Price,Token0 Price (USDC),Token1 Price (WETH),Tick,...,Low (USDC),Close (USDC),Token0 Price Change,Token1 Price Change,Price Ratio,Impermanent Loss (Stable Coin),Impermanent Loss (Token),Virtual Price,Virtual Price Change,Effective Fee
0,2021-01-07 08:00:00,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-18809,1625097600,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC / WETH,6.572546e+18,"1,725,479,688,395,216,800,000,000,000,000,000",2108.334254,0.000474,"199,783",...,2076.132547,2277.090104,0.801199,1.248129,0.641920,0.006110,-0.024070,1.314509e+19,NaN,-1.0
1,2021-01-08 08:00:00,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-18840,1627776000,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC / WETH,8.556401e+18,"1,567,310,799,011,734,800,000,000,000,000,000",2555.341308,0.000391,"197,860",...,2516.882812,2532.374211,1.212019,0.825070,1.468990,-0.004604,0.018207,1.711280e+19,1.301840,-1.0
2,2021-01-09 08:00:00,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-18871,1630454400,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC / WETH,2.028041e+18,"1,280,671,527,367,842,600,000,000,000,000,000",3827.222041,0.000261,"193,820",...,3382.975749,3429.659968,1.497734,0.667675,2.243208,-0.020056,0.076387,4.056082e+18,0.237020,-1.0
3,2021-01-10 08:00:00,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-18901,1633046400,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC / WETH,7.886868e+18,"1,377,695,801,826,872,000,000,000,000,000,000",3307.139370,0.000302,"195,281",...,2961.561786,3000.911862,0.864110,1.157261,0.746685,0.002661,-0.010572,1.577374e+19,3.888909,-1.0
4,2021-01-11 08:00:00,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-18932,1635724800,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC / WETH,1.552800e+19,"1,204,709,674,553,587,300,000,000,000,000,000",4325.082256,0.000231,"192,597",...,4153.451725,4294.180666,1.307802,0.764642,1.710345,-0.008934,0.034956,3.105601e+19,1.968843,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,2023-12-07 08:00:00,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-19550,1689120000,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC / WETH,2.996021e+19,"1,830,921,455,145,474,000,000,000,000,000,000",1872.491003,0.000534,"200,970",...,1866.144593,1878.296738,1.075009,0.930225,1.155645,-0.000654,0.002610,5.992042e+19,0.273678,-1.0
945,2023-12-08 08:00:00,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-19581,1691798400,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC / WETH,5.632675e+19,"1,842,856,289,879,040,800,000,000,000,000,000",1848.316025,0.000541,"201,099",...,1845.694273,1847.153587,0.987089,1.013079,0.974345,0.000021,-0.000084,1.126535e+20,1.880052,-1.0
946,2023-12-09 08:00:00,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-19612,1694476800,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC / WETH,2.513272e+19,"1,985,619,709,190,068,200,000,000,000,000,000",1592.087811,0.000628,"202,592",...,1549.410008,1550.822103,0.861372,1.160938,0.741962,0.002777,-0.011032,5.026543e+19,0.446195,-1.0
947,2023-12-10 08:00:00,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-19642,1697068800,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC / WETH,2.726893e+19,"2,019,724,062,301,352,000,000,000,000,000,000",1538.774881,0.000650,"202,932",...,1522.713992,1565.635886,0.966514,1.034646,0.934149,0.000145,-0.000580,5.453787e+19,1.084997,-1.0


In [3]:
# Adding in Exchange Price, reference CoinGecko
import defi.defi_tools as dft
pools_daily['Date'] = pd.to_datetime(pools_daily['Date'])
pools_daily['Date'] = pools_daily['Date'].dt.date
pools_daily.index = pools_daily['Date']
df = dft.geckoHistorical('ethereum')
df['Date'] = df.index.date
df['Date'] = pd.to_datetime(df['Date'])

df_btc = dft.geckoHistorical('bitcoin')
df_btc['Date'] = df_btc.index.date
df_btc['Date'] = pd.to_datetime(df_btc['Date'])


# Filtering for only the dates we have in our dataset
df = df.loc[df.index.isin(pools_daily['Date'])]
print(df)
print(pools_daily)

df_btc = df_btc.loc[df_btc.index.isin(pools_daily['Date'])]
print(df_btc)


pools_daily['ETH Price CoinGecko'] = df['price']
pools_daily['ETH Market Cap CoinGecko'] = df['market_caps']
pools_daily['ETH Volume CoinGecko'] = df['total_volumes']

pools_daily['BTC Price CoinGecko'] = df_btc['price']
pools_daily['BTC Market Cap CoinGecko'] = df_btc['market_caps']
pools_daily['BTC Volume CoinGecko'] = df_btc['total_volumes']


pools_daily.to_csv('pools_daily_weth_btc.csv', sep=';', encoding='utf-8', index=False)

                  price   market_caps  total_volumes       Date
date                                                           
2021-05-04  3439.855070  3.982580e+11   6.419265e+10 2021-05-04
2021-05-05  3245.663149  3.805657e+11   7.843724e+10 2021-05-05
2021-05-06  3524.562728  4.091914e+11   6.355234e+10 2021-05-06
2021-05-07  3495.075869  4.053559e+11   5.043017e+10 2021-05-07
2021-05-08  3493.534500  4.042846e+11   4.498077e+10 2021-05-08
...                 ...           ...            ...        ...
2024-01-05  2270.977439  2.731093e+11   1.834110e+10 2024-01-05
2024-01-06  2267.087035  2.719364e+11   1.873948e+10 2024-01-06
2024-01-07  2241.510827  2.694896e+11   8.619190e+09 2024-01-07
2024-01-08  2220.151562  2.662401e+11   1.013575e+10 2024-01-08
2024-01-09  2331.660329  2.803157e+11   2.622362e+10 2024-01-09

[981 rows x 4 columns]
                                                          ID        Date  \
Date                                                                

In [ ]:
pools_daily

NameError: name 'pools_daily' is not defined

In [ ]:
import defi.defi_tools as dft
import pandas as pd

total_data = pd.DataFrame()

# ADA, DAI, XRP, WBTC, ETH, SOL
df_link = dft.geckoHistorical('link')['price']

df_wbtc = dft.geckoHistorical('bitcoin')['price']

df_dai = dft.geckoHistorical('dai')['price']

df_eth = dft.geckoHistorical('ethereum')['price']

df_ldo = dft.geckoHistorical('lido-dao')['price']


In [ ]:
df_ldo

date
2021-01-05 00:00:00    1.750891
2021-01-06 00:00:00    1.750891
2021-01-07 00:00:00    1.760411
2021-01-08 00:00:00    1.426809
2021-01-09 00:00:00    1.495403
                         ...   
2023-12-25 00:00:00    2.428220
2023-12-26 00:00:00    2.553696
2023-12-27 00:00:00    2.414790
2023-12-28 00:00:00    2.834848
2023-12-28 00:41:00    2.881094
Name: price, Length: 1089, dtype: float64

In [ ]:
start_date, end_date = df_ldo.index[0], df_ldo.index[-1]

df_wbtc = df_wbtc.loc[(df_wbtc.index >= start_date) & (df_wbtc.index <= end_date)]
df_link = df_link.loc[(df_link.index >= start_date) & (df_link.index <= end_date)]
df_eth = df_eth.loc[(df_eth.index >= start_date) & (df_eth.index <= end_date)]
df_dai = df_dai.loc[(df_dai.index >= start_date) & (df_dai.index <= end_date)]

total_data['LINK Price CoinGecko'] = df_link
total_data['WBTC Price CoinGecko'] = df_wbtc
total_data['DAI Price CoinGecko'] = df_dai
total_data['ETH Price CoinGecko'] = df_eth
total_data['LDO Price CoinGecko'] = df_ldo


total_data

,LINK Price CoinGecko,WBTC Price CoinGecko,DAI Price CoinGecko,ETH Price CoinGecko,LDO Price CoinGecko
date,,,,,
2021-01-05,13.586073,31515.575967,1.002005,1025.654768,1.750891
2021-01-06,14.796036,34082.206049,1.000494,1103.358252,1.750891
2021-01-07,14.988738,36933.520137,1.006038,1208.575093,1.760411
2021-01-08,15.006838,39547.084081,1.000357,1229.471315,1.426809
2021-01-09,13.780041,40815.961855,0.998569,1223.729688,1.495403
...,...,...,...,...,...
2023-12-24,34.851644,43752.030885,0.998909,2310.955663,2.363360
2023-12-25,34.412154,43034.971063,0.998111,2264.911144,2.428220
2023-12-26,34.258915,43638.236302,0.999041,2273.632612,2.553696


In [ ]:
# Corerlation Matrix
corr = total_data.corr()
corr

,LINK Price CoinGecko,WBTC Price CoinGecko,DAI Price CoinGecko,ETH Price CoinGecko,LDO Price CoinGecko
LINK Price CoinGecko,1.000000,0.767976,0.151709,0.822948,0.504376
WBTC Price CoinGecko,0.767976,1.000000,0.212333,0.787986,0.479742
DAI Price CoinGecko,0.151709,0.212333,1.000000,0.140825,0.047370
ETH Price CoinGecko,0.822948,0.787986,0.140825,1.000000,0.748163
LDO Price CoinGecko,0.504376,0.479742,0.047370,0.748163,1.000000


In [ ]:
# Get Pairs with highest correlation
corr_pairs = corr.unstack()
corr_pairs = corr_pairs.sort_values(kind="quicksort", ascending=False)
corr_pairs = corr_pairs[corr_pairs != 1]
corr_pairs

LINK Price CoinGecko  ETH Price CoinGecko     0.822948
ETH Price CoinGecko   LINK Price CoinGecko    0.822948
WBTC Price CoinGecko  ETH Price CoinGecko     0.787986
ETH Price CoinGecko   WBTC Price CoinGecko    0.787986
WBTC Price CoinGecko  LINK Price CoinGecko    0.767976
LINK Price CoinGecko  WBTC Price CoinGecko    0.767976
LDO Price CoinGecko   ETH Price CoinGecko     0.748163
ETH Price CoinGecko   LDO Price CoinGecko     0.748163
LDO Price CoinGecko   LINK Price CoinGecko    0.504376
LINK Price CoinGecko  LDO Price CoinGecko     0.504376
WBTC Price CoinGecko  LDO Price CoinGecko     0.479742
LDO Price CoinGecko   WBTC Price CoinGecko    0.479742
DAI Price CoinGecko   WBTC Price CoinGecko    0.212333
WBTC Price CoinGecko  DAI Price CoinGecko     0.212333
LINK Price CoinGecko  DAI Price CoinGecko     0.151709
DAI Price CoinGecko   LINK Price CoinGecko    0.151709
                      ETH Price CoinGecko     0.140825
ETH Price CoinGecko   DAI Price CoinGecko     0.140825
LDO Price 